In [1]:
cd "E:\Text Mining\PROJECT"

E:\Text Mining\PROJECT


In [2]:
import pandas as p
import numpy as np
import itertools
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import re

In [3]:
def getCleanTweet(text):
    tok = WordPunctTokenizer()
    user_pattern = '@[A-Za-z0-9_]+'
    http_pattern = 'https?://[^ ]+'
    www_pattern = 'www.[^ ]+'
    combined_pattern = '|'.join((user_pattern, http_pattern, www_pattern))
    negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                    "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                    "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                    "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                    "mustn't":"must not"}
    neg_pattern = re.compile('\b(' + '|'.join(negations_dic.keys()) + ')\b')
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pattern, '', bom_removed)
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], stripped)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled).lower()
    cleaned = (" ".join(x for x in tok.tokenize(letters_only) if len(x) > 1)).strip()
    cleaned = ''.join(k + k if sum(1 for i in g) > 1 else k for k, g in itertools.groupby(cleaned))
    return cleaned

In [5]:
data = p.read_csv("tweet.csv", encoding="latin-1", names=["target", "ids", "date", "flag", "user", "text"])
data['clean_text'] = [getCleanTweet(text) for text in data['text']]
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
data.info()

E:\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b' i just received my G8 viola exam.. and its... well... .. disappointing.. :\\..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
E:\Anaconda3\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \\../  \\../'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
target        1600000 non-null int64
ids           1600000 non-null int64
date          1600000 non-null object
flag          1600000 non-null object
user          1600000 non-null object
text          1600000 non-null object
clean_text    1600000 non-null object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [7]:
x_values = data['clean_text'].values
y_values = data['target'].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_values, y_values, test_size=0.3, random_state=0)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(encoding='latin-1',binary=False,min_df = 5 ,stop_words='english')
bi_vector = CountVectorizer(encoding='latin-1',stop_words='english',binary=False, ngram_range=(1,2),min_df = 5)

x_train_vec = vector.fit_transform(x_train)
print(x_train_vec.shape)

x_test_vec = vector.transform(x_test)
print(x_test_vec.shape)


(1120000, 40537)
(480000, 40537)


In [9]:
from sklearn.svm import LinearSVC as svm
svm = svm(C=1, max_iter=1000)
svm.fit(x_train_vec,y_train)
svm.score(x_test_vec, y_test)

E:\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7724708333333333